<a href="https://colab.research.google.com/github/nyayuchika/pcd-coursework-nyayuchika/blob/master/PCD_Assignment01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upsampling

In [1]:
#Generate Matrix
import numpy as np
m = 0
n = 0
m = input("Masukkan banyak baris = ")
n = input("Masukkan banyak kolom = ")
print(f"Banyak baris {m}")
print(f"Banyak kolom {n}")
m = int(m)
n = int(n)
a = np.random.randint(10, size=(m,n))
print(a)

Masukkan banyak baris = 3
Masukkan banyak kolom = 3
Banyak baris 3
Banyak kolom 3
[[7 0 4]
 [4 3 1]
 [4 9 9]]


##Nearest Neighbor

In [2]:
print(a)

[[7 0 4]
 [4 3 1]
 [4 9 9]]


In [3]:
k = 2



##Bilinear

##Bicubic

#Downsampling

In [4]:
#Generate Matrix
import numpy as np
m = 0
n = 0
m = input("Masukkan banyak baris = ")
n = input("Masukkan banyak kolom = ")
print(f"Banyak baris {m}")
print(f"Banyak kolom {n}")
m = int(m)
n = int(n)
b = np.random.randint(10, size=(m,n))
print(b)

Masukkan banyak baris = 4
Masukkan banyak kolom = 4
Banyak baris 4
Banyak kolom 4
[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]


##Max

In [5]:
print(b)
print(b.shape)

[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]
(4, 4)


In [6]:
# k = 2

b_m1 = b[0:2,0:2]
print(b_m1)
max_m1 = np.max(b_m1)
print(max_m1)

[[0 3]
 [5 0]]
5


In [19]:
b_m2 = b[0:2,2:4]
print(b_m2)
max_m2 = np.max(b_m2)
print(max_m2)

[[0 2]
 [6 5]]
6


In [23]:
b_m3 = b[2:5,0:2]
print(b_m3)
max_m3 = np.max(b_m3)
print(max_m3)

[[5 7]
 [5 8]]
8


In [22]:
b_m4 = b[2:5,2:4]
print(b_m4)
max_m4 = np.max(b_m4)
print(max_m4)

[[9 4]
 [9 0]]
9


In [26]:
b_max = np.array([[max_m1,max_m2],[max_m3,max_m4]])
print(f"Matriks Downsampling Max: \n",b_max)

Matriks Downsampling Max: 
 [[9 6]
 [8 9]]


##Median

##Mean

In [ ]:
print(b)
print(b.shape)

[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]]
(4, 4)


In [36]:
# k = 2

b_avg1 = b[0:2,0:2]
print(b_avg1)
b_avg1 = b_avg1.flatten()
mean_avg1 = int(sum(b_avg1)/len(b_avg1))
print(mean_avg1)

[[0 3]
 [5 0]]
2


In [37]:
b_avg2 = b[0:2,2:4]
print(b_avg2)
b_avg2 = b_avg2.flatten()
mean_avg2 = int(sum(b_avg2)/len(b_avg2))
print(mean_avg2)

[[0 2]
 [6 5]]
3


In [38]:
b_avg3 = b[2:5,0:2]
print(b_avg3)
b_avg3 = b_avg3.flatten()
mean_avg3 = int(sum(b_avg3)/len(b_avg3))
print(mean_avg3)

[[5 7]
 [5 8]]
6


In [39]:
b_avg4 = b[2:5,2:4]
print(b_avg4)
b_avg4 = b_avg4.flatten()
mean_avg4 = int(sum(b_avg4)/len(b_avg4))
print(mean_avg4)

[[9 4]
 [9 0]]
5


In [40]:
b_mean = np.array([[mean_avg1,mean_avg2],[mean_avg3,mean_avg4]])
print(f"Matriks Downsampling Mean \n",b_mean)

Matriks Downsampling Mean 
 [[2 3]
 [6 5]]


##Conclusion

In [52]:
print(f"""The result from downsampling method:
Before downsampling: \n{b} \n\nAfter downsampling:
1. Max: \n{b_max} \n\n2. Median: \n\n3. Mean: \n{b_mean}""")

The result from downsampling method:
Before downsampling: 
[[0 3 0 2]
 [5 0 6 5]
 [5 7 9 4]
 [5 8 9 0]] 

After downsampling:
1. Max: 
[[9 6]
 [8 9]] 

2. Median: 

3. Mean: 
[[2 3]
 [6 5]]
